In [ ]:

# loads generative model and tokenizer
# from metaflow import FlowSpec, step, NBRunner, Parameter
# import json
# import yaml
# import pandas as pd
# from datasets import Dataset



ImportError: dlopen(/Users/mgg/dev/projets/fine-tuning/.venv/lib/python3.12/site-packages/tensorflow/python/profiler/internal/_pywrap_profiler.so, 0x0002): tried: '/Users/mgg/dev/projets/fine-tuning/.venv/lib/python3.12/site-packages/tensorflow/python/profiler/internal/_pywrap_profiler.so' (open() failed with errno=1), '/System/Volumes/Preboot/Cryptexes/OS/Users/mgg/dev/projets/fine-tuning/.venv/lib/python3.12/site-packages/tensorflow/python/profiler/internal/_pywrap_profiler.so' (no such file), '/Users/mgg/dev/projets/fine-tuning/.venv/lib/python3.12/site-packages/tensorflow/python/profiler/internal/_pywrap_profiler.so' (open() failed with errno=1)

In [6]:
# %%writefile workflow.py
from metaflow import FlowSpec, step, NBRunner, Parameter
import json
import yaml
import pandas as pd
from datasets import Dataset

import random

# from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
# from peft import LoraConfig, get_peft_model
# from trl import SFTConfig, SFTTrainer
from typing import Union
from tqdm import tqdm
from wordllama import WordLlama
# from sentence_transformers.cross_encoder import CrossEncoder

from tools import create_judgement_prompt, extract_values
from owui_connector.owui import WebUIConnector
from tqdm import tqdm


class FineTuningWorkflow(FlowSpec):
    resume_from_checkpoint = Parameter(
        name="resume_from_checkpoint",
        type=str,
        default=""
    )
    infra = Parameter(
        name="infra",
        default="local",
        type=str
    )
    lr = Parameter(
        name="learning_rate",
        type=float,
        default=""
    )
    n_epochs = Parameter(
        name="n_epochs",
        type=int,
        default=5,
    )
    logging_steps = Parameter(
        name="logging_steps",
        default=50,
        type=int
    )
    train_dataset_dir= Parameter(
        name="train_dataset_dir",
        type=str,
        default="bucket/data/train_dataset.json"
    )
    test_dataset_dir= Parameter(
        name="test_dataset_dir",
        type=str,
        default="bucket/data/test_dataset.json"
    )
    output_dir = Parameter(
        name = "output_dir",
        type=str,
        default="train_result"
    )
    model_name = Parameter(
        name="pre_trained_model_name",
        type=str, 
        default=""
    )
    model_dtype = Parameter(
        name="model_dtype",
        type=str, 
        default=""
    )
    data_prop = Parameter(
        name="training_data_proportion",
        type=float,
        default=.1
    )
    lora_alpha = Parameter(
        name="lora_alpha",
        default=1.0,
        type=float,
    )
    lora_rank = Parameter(
        name="lora_rank",
        default=16,
        type=int
    )
    lora_dropout = Parameter(
        name="lora_dropout",
        default=0.1,
        type=float
    )
    max_new_tokens = Parameter(
        name="max_new_tokens",
        default=100,
        type=int
    )

    def __init__(self):
        self.owui_token, self.owui_url, self.owui_fav_model = self.load_owui_config()

        # move to start ?
        match self.infra:
            # to fill with appropriate parameter selection
            case "onyxia":
                pass
            case "local":
                pass
            case "datalab_gcp":
                pass
            case _:
                raise ValueError(f"Unexpected value for parameter infra : '{self.infra}'. Accepted values are : 'onyxia', 'local' and 'datalab_gcp'.")

    def q_a(self, question):
        return self.chat_pipeline([{
            "role": "user",
            "content": question
        }], max_new_tokens=self.max_new_tokens)[0]["generated_text"][1]["content"]

    def load_owui_config(self):
        with open("conf/conf.yaml", "rt") as f:
            conf = yaml.safe_load(f)
        return conf["OWUI_TOKEN"], conf["OWUI_URL"], conf["OWUI_FAV_MODEL"]

    @step
    def start(self):
        self.next(self.load_pre_trained_model, self.load_tokenizer, self.load_training_dataset)

    @step
    def load_pre_trained_model(self):
        """
        Load the pre-trained model
        """
        self.pre_trained_model = AutoModelForCausalLM.from_pretrained(self.model_name, torch_dtype=self.model_dtype)
        self.next(self.create_peft_model)

    @step
    def load_tokenizer(self):
        """
        Load the tokenizer of the model
        """
        self.tokenizer = AutoTokenizer.from_pretrained(self.model_name, torch_dtype=self.model_dtype)
        self.tokenizer.pad_token = self.tokenizer.eos_token # add a padding token, otherwise it raises an erro
        self.next(self.create_peft_model)

    @step
    def load_training_dataset(self):
        """
        Loads the training the dataset in a json file, and format
        them to be used for chat templates.
        """
        with open(self.train_dataset_dir, "rt") as f:
            train_dataset = json.load(f)

        train_dataset = train_dataset[:int(self.data_prop*len(train_dataset))]
        print(f"Number of acronyms : {len(train_dataset)}")


        all_convs = []
        for each_acro in train_dataset:
            for each_conv in each_acro["conversation"]:
                all_convs.append(each_conv)

        self.raw_conversations = all_convs

        conv_idx_for_test: int = random.randint(0, len(train_dataset)-1) # take one conversation for test
        self.test_conv = train_dataset[conv_idx_for_test]
        print(f"Example of conversation : {self.test_conv}")

        self.next(self.tokenize_training_conversations)
    
    @step
    def tokenize_training_conversations(self):
        """
        Tokenize conversations and loads them in a hugging face
        dataset.
        """
        tokenized_conversations = self.tokenizer.apply_chat_template(
            conversation=self.all_convs,
            return_tensors="pt",
            return_dict=True,
            truncation=True,
            padding=True,
            max_length=256,
        )

        tokenized_conversations["labels"] = tokenized_conversations["input_ids"]
        self.train_dataset: Dataset = Dataset.from_dict(tokenized_conversations)
        self.next(self.create_peft_model)

    @step
    def create_peft_model(self):
        self.peft_config = LoraConfig(
                r=self.lora_rank,
                lora_alpha=self.lora_alpha,
                target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
                lora_dropout=self.lora_dropout,
                bias="lora_only",
                modules_to_save=["decode_head"]
        )

        self.lora_model = get_peft_model(self.pre_trained_model, self.peft_config)
        
        self.next(self.create_trainer, self.create_chat_pipeline)

    @step
    def create_chat_pipeline(self):
        self.chat_pipeline = pipeline("text-generation", model=self.lora_model, tokenizer=self.tokenizer, max_new_tokens=self.max_new_tokens, do_sample=True)
        self.next(self.hot_evaluation)

    @step
    def create_trainer(self):
        # Initialize trainer
        self.training_args = SFTConfig(
            output_dir=self.output_dir,
            # max_steps=100,
            num_train_epochs=self.n_epochs,
            learning_rate=self.lr,
            per_device_train_batch_size=1, # it seems that with a batch size greater than 1, weights are updated with the average gradient loss over
            # all the batch, hence the model could not be updated with the information about a particular element of the dataset.
            # For our usecase, batch size of 1 is better  https://discuss.pytorch.org/t/how-sgd-works-in-pytorch/8060
            logging_steps=self.logging_steps, # doc about what is step vs batch : https://discuss.huggingface.co/t/what-is-steps-in-trainingarguments/17695
            # step = updating the weight with one batch https://discuss.huggingface.co/t/what-is-the-meaning-of-steps-parameters/56411
            # warmup_ratio=.0,
            # save_steps=100,
            # eval_strategy="steps",
            # eval_steps=50,
        )

        self.trainer = SFTTrainer(
            model=self.lora_model,
            args=self.training_args,
            train_dataset=self.train_dataset,
            eval_dataset=self.train_dataset,
            peft_config=self.peft_config,
        )  
        self.next(self.train)
    
    @step
    def train(self):
        if self.checkpoint_path == "":
            self.trainer.train()
        else:
            self.trainer.train(resume_from_checkpoint=self.checkpoint_path)
            
        self.next(self.hot_evaluation)
    
    @step
    def hot_evaluation(self):
        self.lora_model.eval() # eval mode : stops useless gradient computations
        for _ in range(10):
            print(self.q_a(f"What is {self.test_conv["acronym"]} ?"))
            print("--------\n")

    @step
    def load_test_dataset(self):
        with open(str(self.test_dataset_dir), "rt") as f:
            self.test_dataset = json.load(f)

        print(f"Example of element in test dataset : '{self.test_dataset[0]}'")
        self.next(self.create_answer_dataset)
    
    @step
    def create_answer_dataset(self):
        """
        Test the model on the questions of the test dataset.
        Creates self.answer_dataset
        """
        answer_dataset = []

        for each_try in tqdm(self.test_dataset): # todo: use transformers pipeline parallelism
            question = [each_try["conversation"][0][0]]
            answer = self.chat_pipeline(question, pad_token_id=self.tokenizer.eos_token_id, max_new_tokens=200)[0]['generated_text'][1]['content']
            answer_dataset.append({
                "question": question[0]['content'],
                "answer": answer,
                "expected_answer": each_try["conversation"][0][1]['content'],
                "ground_truth": each_try["ground_truth"],
                "acronym": each_try["acronym"]
            })
        self.answer_dataset = pd.DataFrame.from_dict(answer_dataset) # packaging everything in a pandas datafram
        self.next(self.show_exemple_answer)
    
    @step
    def show_exemple_answer(self):
        displayed_examples = random.sample(list(self.answer_dataset.index), 5)
        print(self.answer_dataset.loc[displayed_examples])
        self.next(self.compute_sim_with_static_embedding, self.compute_sim_with_cross_encoder, self.compute_sim_with_llm_as_a_judge)
    
    @step
    def compute_sim_with_cross_encoder(self):
        self.cross_encoder = CrossEncoder("cross-encoder/stsb-distilroberta-base")
        couple_list = self.answer_dataset[["answer", "expected_answer"]].to_numpy().tolist() # not using direct dataframe to use parallel computing of lib sentence_transformer

        res = self.cross_encoder.predict(couple_list)

        self.answer_dataset["cross_encoder_score"] = res
        self.next(self.save_answer_dataset)

    @step
    def compute_sim_with_static_embedding(self):
        # Load pre-trained static embeddings (truncate dimension to 64)
        self.wl = WordLlama.load(trunc_dim=64)

        self.answer_dataset["static_embedding_sim"] = self.answer_dataset.apply(lambda x : self.wl.similarity(x.answer,x.expected_answer), axis="columns")
        self.next(self.save_answer_dataset)


    @step
    def compute_sim_with_llm_as_a_judge(self):
        owui = WebUIConnector(self.owui_token, self.owui_url, fav_model=self.owui_fav_model)
        triplet_list = self.answer_dataset[["question", "answer", "expected_answer"]].to_numpy().tolist()

        all_results = []
        for each_triplet in tqdm(triplet_list):
            prompt = create_judgement_prompt(question=each_triplet[0], answer_to_test=each_triplet[1], definition=each_triplet[2])
            response = owui.get_chat_response(prompt)
            result, explain = extract_values(response)
            if result is None:
                result = 0
            all_results.append({"result": result, "explain": explain})

        self.answer_dataset["llm_judge_result"] = pd.Series([each_res["result"] for each_res in all_results], dtype="int")
        self.answer_dataset["llm_judge_eplain"] = [each_res["explain"] for each_res in all_results]
        self.acc_from_judge = self.answer_dataset.llm_judge_result.sum()/self.answer_dataset.shape[0] # fine tuned model on more epochs
        print("Accuracy according to LLM judge :", self.acc_from_judge)
        self.next(self.save_answer_dataset)

    @step
    def save_answer_dataset(self):
        self.answer_dataset.to_csv(f"test_dataset_{random.randint(0, 15542)}")

    @step   
    def end(self):
        return


